 ### main_prime 
accepts 2 file names:1st for the JD,2nd for the resume. This is the start function for:
* loading and cleaning the texts
* generating a structure for each doc {'word':w,'guid':'','index':idx,'related_id':[],'sent_no':sent_no,'replaced':'','syn_list':[],'syn':0,'lemma':'','stemm':'','pos':''}) where realted_id is the id in the other document whith matches, and replaced is the suggested word for replacment, syn_list the list of sysnonyms founf for the word and pos the point of speech tags 

#### import_packages 
shows which packages are used and imported - depending on the installation dependencies have to be taken in condieration before!

In [1]:
    #!pip install lemminflect
    #!pip install varname
    import pickle
    import time
    import pandas as pd
    import re
    #from gensim import summarization
    import nltk
    from nltk.corpus import stopwords
    from nltk.tokenize import word_tokenize 
    from nltk.tokenize import sent_tokenize
    import string
    import spacy
    from nltk.corpus import wordnet
    from varname import nameof
    from nltk.stem import LancasterStemmer
    from nltk.stem import PorterStemmer
    from nltk.stem import WordNetLemmatizer
    from lemminflect import getInflection
    import gensim
    import gensim.downloader as api
    from gensim.test.utils import common_texts
    from gensim.models import Word2Vec


In [3]:
def main_prime(jd_file,res_file):
    import time
    t0=time.time()
    #!pip install lemminflect
    #!pip install varname
    import pickle
    
    import pandas as pd
    import re
    #from gensim import summarization
    import nltk
    from nltk.corpus import stopwords
    from nltk.tokenize import word_tokenize 
    from nltk.tokenize import sent_tokenize
    import string
    import spacy
    from nltk.corpus import wordnet
    from varname import nameof
    from nltk.stem import LancasterStemmer
    from nltk.stem import PorterStemmer
    from nltk.stem import WordNetLemmatizer
    from lemminflect import getInflection
    import gensim
    import gensim.downloader as api
    from gensim.test.utils import common_texts
    from gensim.models import Word2Vec
    print('all packages loaded')
    #import_packages()
    new_wv=load_word_vector()
    print('word vec loaded')
    #cleaning and preparing the list of dicts structure for both docs
    res=prep_all(jd_file)
    jd=prep_all(res_file)                           
    print('basic dict structure generated')
    #check perfect matches of  lemmas in RES
    result=match_word_sets(jd,res)
    perfect_match_lst=result[1]
    remaining_lst=result[2]
    print('1:1 word matches generated')
    # search for synonyms of lemmas in JD and in RES and write to list of dict structure
    jd_syns=insert_syns(result[3],remaining_lst,new_wv)
    res_syns=insert_syns(result[0],remaining_lst,new_wv)
    
    # find matches in RES for word synonyms in JD and log it to the dicts
    x=match_syns(jd_syns,res_syns,remaining_lst)
    print('synonym matches generated')
    # basic metrics
    perf_match=[]
    no_match=[]
    syn_match=[]
    n=0
    for i in x[1]:

        for j in i:
            n+=1
            if j['syn']==1.0:
                l=[]
                l.append(j['word'])
                l.append(j['index'])
                l.append(j['syn'])
                l.append(j['related_id'])
                l.append(j['replaced'])
                perf_match.append(l)
            elif j['syn']==0:
                l=[]
                l.append(j['word'])
                l.append(j['index'])
                l.append(j['syn'])
                l.append(j['related_id'])
                l.append(j['replaced'])
                no_match.append(l)
            else:
                l=[]
                l.append(j['word'])
                l.append(j['index'])
                l.append(j['syn'])
                l.append(j['related_id'])
                l.append(j['replaced'])
                syn_match.append(l)


    print('No of Synmatches',len(syn_match))
    print(syn_match,'\n')
    print('No of Nomatches',len(no_match))
    print(no_match,'\n')
    print('No of Perfectmatches',len(perf_match))
    print(perf_match,'\n')
    print('No total',n)
    t1=time.time()
    print('time needed:', t1-t0,' sec \n')

    base_rate=len(perf_match)/n
    primed_rate=(len(perf_match)+len(syn_match))/n

    print('From ', base_rate, ' to ', primed_rate)
    return(x)





#### Some internal functions for support processes are listed

In [4]:
#------------------------------------------------------------------------------------------------------------
# runs the first text preparations step by step by calling the support functions 
#------------------------------------------------------------------------------------------------------------
def prep_all (jd_file):
    
    jd=jd_file.split('.')
    jd_txt=open(jd_file).read()
    
    jd_cl=clean_text(jd_txt,False)
    jd_sent=sent_tokenize(jd_cl)
 
    
    #pickle the sentence tokens
    outfile = open('jd_sent_tok.pkl','wb') 
    pickle.dump(jd_sent,outfile)
    outfile.close()     
    
    stop_words = set(stopwords.words('english'))
    noun_tags=['NN', 'NNS','NNP','NNPS']
    verb_tags=['VB','VBG','VBD','VBN','VBP','VBZ']
    
   
    iw=index_words(jd_sent,'list') # base indexing
    iw_nostops=[]  
    for s in iw: 
        l=remove_stopwords(s,{'.'})
        l_tagged=tag_words(l)
        iw_nostops.append(l_tagged)
       
    return iw_nostops



In [5]:
#------------------------------------------------------------------------------------------------------------
def load_word_vector():
    corpus = api.load('text8')
    model = gensim.models.Word2Vec(sentences=corpus, size=100, window=5, min_count=1, workers=4)
    new_wv=model.wv
    return(new_wv)
#------------------------------------------------------------------------------------------------------------
#recursisive deletion of multi characters ch1 in string t
#------------------------------------------------------------------------------------------------------------
def multi_del_recurse(t,ch1):
    ch=ch1+ch1
    if re.search(ch, t)!= None :
        t1=re.sub(r'\\','','\.')
        t=re.sub(ch,t1,t)
        return(multi_del_recurse(t,ch1))
    else:
        return t
    
#------------------------------------------------------------------------------------------------------------
#text cleaing with parameter. If punctuation then sentence delimiters are wipped too
#------------------------------------------------------------------------------------------------------------
def clean_text(text,punctuation):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    if punctuation:
        text = re.sub('\[.*?!\]', ' ', text)#remove !
        text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)#remove other punctuations
    else:
        text = text.lower() #to lower
        text = re.sub('\w*\d\w*', '', text) #clearing digits
        '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
        text = re.sub('[\n*]', ' . ', text)
        text = re.sub('[(,)&,%,$,§]', ' . ', text)
        text = re.sub('[/‘’“”…]', '', text)
        text = re.sub('\u20AC', '', text)#remove € 
        text = re.sub('\xa0', ' ', text)
        text=multi_del_recurse(text,'\.')
    return text# indexing the words in sentences with running index numbers and returns a list of dicts or json

#------------------------------------------------------------------------------------------------------------
# generating the basic dict structure and the indexing of the docs
#------------------------------------------------------------------------------------------------------------
def index_words(list_of_sent_tokens,ret_mode):
    import json
    set_idx=set()
    lst=[]
    idx=1
    sent_no=1
    for s in list_of_sent_tokens:
        tok=word_tokenize(s)
        idx_lst=[]
        for w in tok:
            idx_lst.append({'word':w,'guid':'','index':idx,'related_id':[],'sent_no':sent_no,'replaced':'','syn_list':[],'syn':0,'lemma':'','stemm':'','pos':''})
            idx +=1
        lst.append(idx_lst)
        sent_no +=1
    # Serializing json
    json_object = json.dumps(lst, indent = 4)
    json_object= re.sub(r'\[', '{', json_object)
    json_object= re.sub(r'\]', '}', json_object)
   
    if ret_mode=='list': return lst 
    else:return json_object
#------------------------------------------------------------------------------------------------------------
#finds a dict entry by idx as index
#------------------------------------------------------------------------------------------------------------
def get_dict_by_idx(list_of_dicts,idx,replace,new_word):
    for s in list_of_dicts:
        for w in s:
            if w['index']==idx:
                if replace: w['replaced']=new_word
                return w    
    return{}

#------------------------------------------------------------------------------------------------------------
#finds a dict by word as value. lemmas are compared!
#------------------------------------------------------------------------------------------------------------
def get_dict_by_word(list_of_dicts,word):
    result=[]
    for d in list_of_dicts:
        result.extend([element for element in d if element['lemma'] == word])
    return result

#------------------------------------------------------------------------------------------------------------
# fills the stemm entries
#------------------------------------------------------------------------------------------------------------
def replace_word_stemm(list_of_dicts,stemm,new_word):
    for s in list_of_dicts:
        for w in s:
            if w['stemm']==stemm:
                w['replaced']=new_word
                return w    
    return{}
#------------------------------------------------------------------------------------------------------------
# cleans stop_words by pattern
#------------------------------------------------------------------------------------------------------------
def remove_stopwords(dict_list,pattern):
        stop_words = set(stopwords.words('english'))
        if pattern!={}:
            stop_words=stop_words.union(pattern)
        dict_list[:]= [x for x in dict_list if x['word'] not in stop_words]
        return dict_list


#------------------------------------------------------------------------------------------------------------
# wiping multiples in an iterable and deliveres unique entries only
#------------------------------------------------------------------------------------------------------------
def getUniqueItems(iterable):
    result = []
    for item in iterable:
        if item not in result:
            result.append(item)
    return result

#------------------------------------------------------------------------------------------------------------
# checks for 1:1 word matches (lemmas!) and fills it in 'replace':'value' and cross-reference it in related_id
#------------------------------------------------------------------------------------------------------------
def match_word_sets(base_dict_lst,target_dict_lst): #gets lists of indexed dicts
    base_lst=gen_wlist_from_dlist(base_dict_lst)
    target_lst=gen_wlist_from_dlist(target_dict_lst)
    s1=set(base_lst[0])
    s2=set(target_lst[0])
    s_match=s1&s2
    s_remaining=s1-s_match
    all_tags=['VB','VBG','VBD','VBN','VBP','VBZ','NN', 'NNS','NNP','NNPS']
    for sent_jd in base_dict_lst:
        for w_jd in sent_jd:
            if w_jd['pos']in all_tags:
                for sent_res in target_dict_lst:
                    for w_res in sent_res:
                        if (w_res['pos']in all_tags) and (w_jd['lemma']== w_res['lemma']):
                            w_res['replaced']=w_jd['lemma']
                            w_res['syn']=1.0
                            if not w_jd['index'] in w_res['related_id']:w_res['related_id'].append(w_jd['index'])
                            if not w_res['index'] in w_jd['related_id']:w_jd['related_id'].append(w_res['index'])
  
    return [target_dict_lst,list(s_match),list(s_remaining),base_dict_lst]
#------------------------------------------------------------------------------------------------------------
# replaces multiple synonyms in the target structure
#------------------------------------------------------------------------------------------------------------
def replace_synonyms(syn_tuple,target_dict):
    count=0
    for s in target_dict:
        for w in s:
            
            for st in syn_tuple['syns']:
                
                if w['lemma'] == st[0]:
                    w['replaced']=getInflection(syn_tuple['word'],tag=w['pos'])[0]
                    
                    #testing
                    #w['word']=getInflection(syn_tuple['word'],tag=w['pos'])[0]
                    
                    w['syn']=st[1]
                    count +=1
    
    return (target_dict,count)   

#------------------------------------------------------------------------------------------------------------
# converts a dict list to a word list - only verbs and nouns are filtered 
#------------------------------------------------------------------------------------------------------------
def gen_wlist_from_dlist(dlist):
    all_tags=['VB','VBG','VBD','VBN','VBP','VBZ','NN', 'NNS','NNP','NNPS']
    #all_tags=['VB','VBG','VBD','VBN','VBP','VBZ']
    #print(dlist)
    wlist=[[],[]]
    l=[]
    ind_l=[]
    for d in dlist:
        for w in d:
            if w['pos'] in all_tags:
                l.append(w['lemma'])
                ind_l.append(w['index'])
        if l!=[]:
            wlist[0].extend(l)
            wlist[1].extend(ind_l)
    return wlist

#------------------------------------------------------------------------------------------------------------
# generates a dict struture with word and it's sysnonyms using a word vector
#------------------------------------------------------------------------------------------------------------
def create_syn_dict(word,idx,new_wv):# with w2vec this needs some time!
      
    syn_list=[]
    i=1
    
    synonyms=[]
    if word in new_wv:    #'WORD VECTOR'        
        synonyms=new_wv.most_similar(word)
    
    for s in synonyms:
        lem=get_lemma(s[0])
        if lem!=s[0]:
            s=(lem,s[1]) 


    dict={'word':word,'index':idx,'syns':synonyms}
        
    return dict

#------------------------------------------------------------------------------------------------------------
# searching synonyms for remaining jd words and save in 'syn_list'
#------------------------------------------------------------------------------------------------------------
def insert_syns(dict,word_lst,new_wv):
    
    all_syns=[]
    for i in dict:
        for j in i:
            if j['lemma']in word_lst:
                syn_tuple=(create_syn_dict(j['lemma'],j['index'],new_wv))
                j['syn_list'].extend(syn_tuple['syns'])
   
    return(dict)
            
#------------------------------------------------------------------------------------------------------------
# converts tagging system treebank to wordnet
#------------------------------------------------------------------------------------------------------------
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''
#------------------------------------------------------------------------------------------------------------
# getting lemma of a word
#------------------------------------------------------------------------------------------------------------
def get_lemma(word):
    lemmatizer=WordNetLemmatizer()
    tag=get_pos_tag(word)
    tag=get_wordnet_pos(tag)
    if tag=='':tag='n'
    lemmatized_word=lemmatizer.lemmatize(word,pos=tag)
    return(lemmatized_word)

#------------------------------------------------------------------------------------------------------------
# getting the POS Tag of a word
#------------------------------------------------------------------------------------------------------------
def get_pos_tag(word):
    return nltk.pos_tag([word])[0][1]

#------------------------------------------------------------------------------------------------------------
# fill all tags, stems and lemmas into a dict struct
#------------------------------------------------------------------------------------------------------------
def tag_words(dict_list):
    porter = PorterStemmer()
    if dict_list==[]: return []
    for w in dict_list:
        w['pos']=nltk.pos_tag([w['word']])[0][1]
        w['stemm']=porter.stem(w['word'])
        w['lemma']=get_lemma(w['word'])
    return dict_list

#------------------------------------------------------------------------------------------------------------
# matching syns of JD words without 1:1 match held in word_lst in the Resume and modifies both dict structures
#------------------------------------------------------------------------------------------------------------
def match_syns(jd_dict,res_dict,word_lst):#word_lst are lemmas from JD with no 1:1 match
    synonyms_in_res=[]
    for w in word_lst:
        match=get_dict_by_word(jd_dict,w)
        if (match!=[]) and (match[0]['syn_list']!=[]):
            sl=match[0]['syn_list']
            for syn in sl:
                syn_match=get_dict_by_word(res_dict,syn[0])
                #print(syn,' we found in RES: ',syn_match,'\n')
                if syn_match!=[] and syn_match[0]['syn']!=1.0:
                    syn_match[0]['replaced']=syn[0]    
                    syn_match[0]['syn']=syn[1]
                    syn_match[0]['related_id'].append(match[0]['index'])
                    match[0]['related_id'].append(syn_match[0]['index'])
                    synonyms_in_res.append(syn_match)
                    
    return([jd_dict,res_dict])

#------------------------------------------------------------------------------------------------------------
# preparing a list struture of the resulting dict structures for pandas and saving in Excel
#------------------------------------------------------------------------------------------------------------
def prep_list(jd_dict,res_dict):
    pdata={'A-JD orig word':[],'B-JD Lemma of A':[],'C-IDX in JD':[],'D-related IDX in RES':[],'E-Matches in RES':[],'F-Original Words in RES':[],'G-Inflect':[]}
    
    for i in jd_dict:
        for j in i:
            pdata['A-JD orig word'].append(j['word'])
            pdata['B-JD Lemma of A'].append(j['lemma'])
            pdata['C-IDX in JD'].append(j['index'])
            pdata['D-related IDX in RES'].append(j['related_id'])
            el=''
            res_words=''
            infl=''
            if j['related_id']!=[]:
                for idx in j['related_id']:
                    r_w=get_dict_by_idx(res_dict,idx,False,'')
                    res_words+=r_w['word']+'|'
                    infl=getInflection(r_w['lemma'],r_w['pos'])
                    if r_w['syn']==1.0:
                        if el=='':
                            el='match: '
                            el+=str(r_w['lemma'])
                    elif r_w['syn']!=0:
                                if el=='': 
                                    el='synonym: '
                                    el+=str(r_w['replaced'])
            pdata['E-Matches in RES'].append(el)
            pdata['F-Original Words in RES'].append(res_words)
            pdata['G-Inflect'].append(infl)

    df=pd.DataFrame.from_dict(pdata)
# Write to file
    with pd.ExcelWriter('the_unbelievable_list.xlsx') as writer:
        df.to_excel(writer, sheet_name='data')            

    return df




In [6]:
doc_dicts=main_prime("Z:\SEMP_JD.txt","Z:\SEMP_RES1.txt")
print(doc_dicts)

all packages loaded
word vec loaded
basic dict structure generated
1:1 word matches generated
synonym matches generated
No of Synmatches 22
[['positioned', 57, 0.6173942685127258, [95], 'position'], ['maintain', 115, 0.7605767250061035, [333, 306], 'maintain'], ['evaluating', 126, 0.7597982883453369, [151], 'evaluate'], ['efficiency', 128, 0.7839260697364807, [291, 135, 122], 'efficiency'], ['designed', 157, 0.6399011611938477, [110], 'design'], ['cost', 160, 0.7667965888977051, [122], 'cost'], ['metrics', 171, 0.6069971919059753, [184], 'metric'], ['indicators', 175, 0.725570797920227, [291], 'indicator'], ['enhancements', 190, 0.6961054801940918, [291], 'enhancement'], ['regulations', 197, 0.7029350996017456, [456], 'regulation'], ['restrictions', 199, 0.6076983213424683, [263, 108], 'restriction'], ['coach', 231, 0.7942534685134888, [71], 'coach'], ['mentor', 233, 0.6634506583213806, [16], 'mentor'], ['user', 257, 0.6169933080673218, [320], 'user'], ['organisation', 258, 0.867704033

###### POS TAG LIST
POS tag list:

    CC coordinating conjunction
    CD cardinal digit
    DT determiner
    EX existential there (like: "there is" ... think of it like "there exists")
    FW foreign word
    IN preposition/subordinating conjunction
    JJ adjective 'big'
    JJR adjective, comparative 'bigger'
    JJS adjective, superlative 'biggest'
    LS list marker 1)
    MD modal could, will
    NN noun, singular 'desk'
    NNS noun plural 'desks'
    NNP proper noun, singular 'Harrison'
    NNPS proper noun, plural 'Americans'
    PDT predeterminer 'all the kids'
    POS possessive ending parent's
    PRP personal pronoun I, he, she
    PRP$ possessive pronoun my, his, hers
    RB adverb very, silently,
    RBR adverb, comparative better
    RBS adverb, superlative best
    RP particle give up
    TO to go 'to' the store.
    UH interjection errrrrrrrm
    VB verb, base form take
    VBD verb, past tense took
    VBG verb, gerund/present participle taking
    VBN verb, past participle taken
    VBP verb, sing. present, non-3d take
    VBZ verb, 3rd person sing. present takes
    WDT wh-determiner which
    WP wh-pronoun who, what
    WP$ possessive wh-pronoun whose
    WRB wh-abverb where, when
